In [3]:
import numpy as np

MAX_INVENTORY = 20
ORDER_COST = 5
HOLDING_COSTS = [1, 2]  # holding costs per product
DEMAND_PROB = 0.5  
TARGET_INVENTORY = 5  # reorder level
REORDER_THRESHOLD = 1  # reorder when inventory reaches this threshold



In [6]:
states = [(i, j) for i in range(1, MAX_INVENTORY + 1) for j in range(1, MAX_INVENTORY + 1)]

# Fixed Policy Function
def fixed_policy(state):
    inventory1, inventory2 = state
    order1 = max(0, TARGET_INVENTORY - inventory1) if inventory1 == REORDER_THRESHOLD else 0
    order2 = max(0, TARGET_INVENTORY - inventory2) if inventory2 == REORDER_THRESHOLD else 0
    return order1, order2

# Transition function
def transition(state, action):
    inv1, inv2 = state
    order1, order2 = action
    outcomes = [(0, 0, (1 - DEMAND_PROB) ** 2),
                (0, 1, DEMAND_PROB * (1 - DEMAND_PROB)),
                (1, 0, DEMAND_PROB * (1 - DEMAND_PROB)),
                (1, 1, DEMAND_PROB ** 2)]
    
    next_states = {}
    for d1, d2, prob in outcomes:
        new_inv1 = min(inv1 + order1 - d1, MAX_INVENTORY)
        new_inv2 = min(inv2 + order2 - d2, MAX_INVENTORY)
        next_state = (max(1, new_inv1), max(1, new_inv2))
        if next_state in next_states:
            next_states[next_state] += prob
        else:
            next_states[next_state] = prob
    return next_states

# Cost function
def cost(state):
    return state[0] * HOLDING_COSTS[0] + state[1] * HOLDING_COSTS[1]

In [7]:

# Simulation under Fixed Policy
def simulate_fixed_policy(time_steps=10000):
    state = (TARGET_INVENTORY, TARGET_INVENTORY)
    total_cost = 0

    for _ in range(time_steps):
        holding_cost = cost(state)
        total_cost += cost(state)
        
         # Apply fixed policy
        action = fixed_policy(state)
        if action != (0, 0):
            total_cost += ORDER_COST

        # Calculate next state based on demand
        next_states = transition(state, action)
        state = max(next_states, key=next_states.get)  # select most probable state

    return total_cost / time_steps

print("Average cost under fixed policy:", simulate_fixed_policy())


Average cost under fixed policy: 15.0


In [ ]:
# limiting distribution?


# Value Iteration for Bellman Equation
def value_iteration_poisson():
    V = {s: 0 for s in states}
    delta = float('inf')
    epsilon = 1e-6 
    
    while delta > epsilon:
        delta = 0
        for state in states:
            # Expected cost calculation here
            action_costs = []
            for action in [(0, 0), (TARGET_INVENTORY - state[0], TARGET_INVENTORY - state[1])]:
                expected_cost = cost(state) + (ORDER_COST if action != (0, 0) else 0)
                
                # Transition probabilities and update
                next_states = transition(state, action)
                expected_cost += sum(prob * V[next_state] for next_state, prob in next_states.items())
                
                action_costs.append(expected_cost)
            
            min_cost = min(action_costs)
            delta = max(delta, abs(min_cost - V[state]))
            V[state] = min_cost

    return V


print("Optimal average cost via value iteration:", min(value_iteration_poisson().values()))
